In [ ]:
import ee
import geemap
import os
import numpy as np
try:
    import rasterio
except ImportError:
    print("Warning: rasterio not installed. Min/Max values will be defaulted.")
from IPython.display import display

# Initialize Earth Engine
try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()

# --- 1. DEFINE PATH & CALCULATE DYNAMIC STATISTICS ---
geotiff_path = 'chlorophyll_smooth.tif'

# We calculate the REAL min/max from the file so the colors are accurate
try:
    with rasterio.open(geotiff_path) as src:
        data = src.read(1)
        nodata = src.nodata
        if nodata is None:
            nodata = -9999
            
        # Mask out nodata values to find the real range
        valid_data = data[data != nodata]
        valid_data = valid_data[~np.isnan(valid_data)]
        
        min_value = float(np.min(valid_data))
        max_value = float(np.max(valid_data))
        
        print(f"✅ Detected Data Range: {min_value:.2f} to {max_value:.2f} mg/m³")
except Exception as e:
    print(f"⚠️ Could not read stats from file ({e}). Using defaults.")
    min_value = 0
    max_value = 15

# --- 2. SETUP BACKGROUND CONTEXT (Sentinel-2) ---
# Define area of interest (Generic Bounding Box for context)
aoi = ee.Geometry.Polygon([[[-45.559114, -18.954365], [-45.559114, -18.212409], 
                           [-44.839706, -18.212409], [-44.839706, -18.954365], 
                           [-45.559114, -18.954365]]])

# Get the water mask from Sentinel-2 data (Optional context)
sentinel2 = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
    .filterBounds(aoi) \
    .filterDate('2020-01-01', '2020-04-01') \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
    .median()

MNDWI = sentinel2.normalizedDifference(['B3', 'B11'])
water_mask = MNDWI.gt(0.3)

# --- 3. CREATE MAP ---
Map = geemap.Map()
Map.add_basemap('SATELLITE')

# Center map specifically on the new raster
try:
    Map.centerObject(geemap.raster_bounds(geotiff_path), 13)
except:
    Map.centerObject(aoi, 10)

# Add the Smooth Interpolated Raster
Map.add_raster(
    geotiff_path,
    palette=[
        '#0000ff', '#00ffff', '#00ff00', '#ffff00', '#ff7f00', '#ff0000',
        '#8b0000', '#800080', '#ff00ff', '#8b4513', '#000000'
    ],
    vmin=min_value,
    vmax=max_value,
    nodata=-9999,
    layer_name='Predicted Chlorophyll (Smooth)',
    opacity=0.8  # Slight transparency to see satellite text/details
)

# Add Sentinel-2 water mask (for comparison)
Map.addLayer(
    water_mask, 
    {'min': 0, 'max': 1, 'palette': ['black', 'blue']}, 
    'Sentinel-2 Water Mask',
    False  # Hidden by default
)

Map.addLayer(
    aoi, 
    {'color': 'white', 'width': 2, 'fillColor': '00000000'}, 
    'AOI Boundary'
)

Map.addLayerControl()

# --- 4. DYNAMIC LEGEND ---
def add_legend(map_obj, title, palette, min_val, max_val):
    legend_html = f"""
    <div style='padding: 10px; background-color: white; border-radius: 5px; bottom: 30px; right: 10px; box-shadow: 0 0 5px rgba(0,0,0,0.3);'>
        <h4 style='margin-top:0'>{title}</h4>
        <div style='display: flex; align-items: center;'>
            <span style='font-size: 12px'>{min_val:.1f}</span>
            <div style='flex-grow: 1; height: 20px; width: 200px; background: linear-gradient(to right, {", ".join(palette)}); margin: 0 10px;'></div>
            <span style='font-size: 12px'>{max_val:.1f}</span>
        </div>
        <div style='text-align: center; font-size: 10px; margin-top: 5px;'>mg/m³</div>
    </div>
    """
    map_obj.add_html(legend_html)

add_legend(
    Map, 
    'Predicted Chlorophyll',
    [
        '#0000ff', '#00ffff', '#00ff00', '#ffff00', '#ff7f00', '#ff0000',
        '#8b0000', '#800080', '#ff00ff', '#8b4513', '#000000'
    ],
    min_value,
    max_value
)

display(Map)

Map(center=[-18.58345884758661, -45.1994100000001], controls=(WidgetControl(options=['position', 'transparent_…